# 准备工作

In [86]:
'''
阿里研究院
阿里健康
阿里巴巴商学院
阿里数据

腾讯金融科技
腾讯研究院
腾讯媒体研究院
腾讯云启研究院
酷鹅用户研究院
'''
公众号 = "中大南方文传体育部"

In [37]:
fn = { "output" : { "公众号_htm_snippets": "data_raw_src/公众号_htm_snippets_{公众号}.tsv",
                    "公众号_df": "data_raw_src/公众号_df_{公众号}.tsv",
                    "公众号_xlsx": "data_sets/公众号_url_{公众号}.xlsx" } \
      }

# 采集公众号（requests）

In [7]:
# 目标url

import time
import requests
import pandas as pd
import csv


url = "https://mp.weixin.qq.com/cgi-bin/appmsg"

# 使用Cookie，跳过登陆操作
headers = {
  "Cookie": "pgv_pvi=7968112640; RK=dAxoOkmNPM; ptcz=5e911a8738788675abb010f85db7a6c13b0570811db08ebefce3d01adfff5127; tvfe_boss_uuid=35a8107e1f02cbf7; pgv_pvid=160875890; o_cookie=576726974; pgv_si=s7389809664; uin=o0576726974; skey=@biD9HkZPV; ptisp=ctc; ua_id=o954FR4YG9aIylgbAAAAAAG1mFSkee-v8r6HcCv6PUM=; cert=xdo33QVRQxDDEATQOxVPYZVSl8hcgbBw; noticeLoginFlag=1; xid=; openid2ticket_omd36wPk8CgeWUiAccw6ZoK6cfKM=V9b+6gaDppAhuzkBsqQ2cqmVOoVbPExAl/fu+rqQ2xk=; mm_lang=zh_CN; rewardsn=; wxtokenkey=777; uuid=3f563116b647ce9a578c8d004d3508a8; ticket=a4fe0acfed322dd3eff3a301d94afded6dac3be0; ticket_id=gh_b8ce1853cd9b; rand_info=CAESIIpFhkwqorHCev9Vkx8d1xoS/h28VHdI65d1I+hS8rYh; slave_bizuin=3252904960; data_bizuin=3509374721; bizuin=3252904960; data_ticket=6z/ln+jO+jKPoA5MKzm3D6Cbfkg41cD9D6v6HTaHq8//2C9Hk73W6LtQlNMXteTT; slave_sid=eWpTNFJYU0hHQUFtcUt1TnBGSEZDYU55bHJXX2hhTTlZa3hvcno4MFRNaGd2QUlIT2Vxb0xHUmRWc1RQV2txQkI3OXFlUXhuOWY1aGVYbWdvTGFpNzNhTTdNaGhaRWNWNmwzTmFMRWlEa25Pa0VkZGpwVGkxNTRUdmRTbEtpbGF2UFUwbEZJR0lHdnlBcDFs; slave_user=gh_b8ce1853cd9b; openid2ticket_oAc8s0T6rPCr8qnMO7SoANf_3jVI=LJrk/Pum+BnDCZRPacwc+RgFKJv/+2uJEUtvOa1gdTU=",
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"}

data = {
    "token": "1119521086",
    "lang": "zh_CN",
    "f": "json",
    "ajax": "1",
    "action": "list_ex",
    "begin": "0",
    "count": "5",
    "query": "",
    "fakeid": "MzI0Nzc3MTQyMw==",
    "type": "9",
}



content_list=[]

for i in range(5):
    data["begin"] = i*5
    print(data["begin"])
    time.sleep(3)
    # 使用get方法进行提交
    content_json = requests.get(url, headers=headers, params=data).json()
#     print(content_json)
    # 返回了一个json，里面是每一页的数据
    for item in content_json["app_msg_list"]:
    # 提取每页文章的标题及对应的url
        items = []
        items.append(item["title"])
        items.append(item["link"])
        items.append(item["create_time"])
        content_list.append(items)


name=['title','link','create_time']
test=pd.DataFrame(columns=name,data=content_list)
with pd.ExcelWriter(fn["output"]["公众号_xlsx"].format(公众号="蚂蚁金服科技_requests")) as writer:
    test.to_excel(writer)

# test.to_csv("../微信公众号爬虫_zhichao/南方周末.csv",mode='a',encoding='utf-8')
# print("保存成功")

0
5
10
15
20


NameError: name 'ExcelWriter' is not defined

# 采集公众号（selenium）

In [19]:
import pandas as pd
import numpy as np
from lxml.html import fromstring
import time
from random import random

# when selenium main_content is used
# Parses an HTML document from a string constant.  Returns the root nood
# root = fromstring(df.loc[1,"html_snippets"]) 

## 使用Selenium
* 要更改 opts.binary_location 至自己本地的Chrome浏览器，建议portable
* Chrome浏览器 和 chromedriver.exe要同版本号到小数后一位
* 要确保可以 开启浏览器机器人
* 要确保浏览器机器人 可以打开网页 driver.get("https://mp.weixin.qq.com")

In [87]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

#caps=dict()
#caps["pageLoadStrategy"] = "none"   # Do not wait for full page load

opts = webdriver.ChromeOptions()
opts.add_argument('--no-sandbox')#解决DevToolsActivePort文件不存在的报错
opts.add_argument('window-size=1920x3000') #指定浏览器分辨率
opts.add_argument('--disable-gpu') #谷歌文档提到需要加上一这个属性来规避bug
opts.add_argument('--hide-scrollbars') #隐藏滚动条, 应对些特殊页面
#opts.add_argument('blink-settings=imagesEnabled=false') #不加载图片, 提升速度
#opts.add_argument('--headless') #浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败

opts.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" #"H:\_coding_\Gitee\InternetNewMedia\CapstonePrj2016\chromedriver.exe"  

# "H:\_coding_\Gitee\InternetNewMedia\CapstonePrj2016\chromedriver.exe"  
driver = webdriver.Chrome( chrome_options = opts) #desired_capabilities=caps,

F:\qq\python\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


In [88]:
driver.get("https://mp.weixin.qq.com")

## 填表登入

selenium 的定位方法
* find_element_by_id &ensp;&ensp;&ensp;  根据标签id定位
* find_element_by_name   &ensp;&ensp;&ensp; 根据标签的name定位
* find_element_by_xpath  &ensp;&ensp;&ensp; 根据xpath定位
* find_element_by_link_text  &ensp;&ensp;&ensp; 通过文字链接来定位元素
* find_element_by_partial_link_text  &ensp;&ensp;&ensp;  通过文字链接来定位元素
* find_element_by_tag_name  &ensp;&ensp;&ensp;  根据标签的名字定位
* find_element_by_class_name  &ensp;&ensp;&ensp; 通过class name 定位
* find_element_by_css_selector  &ensp;&ensp;&ensp;  根据元素属性来定位

In [89]:
payload =  {"account": "2646977745@qq.com", "password": "as83373734"}

driver.find_element_by_xpath('//div[@class="login__type__container login__type__container__scan"]/a').click()

WebDriver 常用方法：
* clear()清楚文本
* send_keys(values)模拟按键输入
* click()模拟点击
* submit模拟提交

In [90]:
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="account"]').clear()
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="account"]').send_keys(payload['account'])
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="password"]').clear()
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="password"]').send_keys(payload['password'])

In [91]:
driver.find_element_by_xpath('//div[@class="login_btn_panel"]/a').click()

## 点选单

其他常用方法
* size：返回元素的尺寸
* text：获取元素的文本
* get_attribute：获取属性值  &ensp;&ensp;&ensp; get_attribute('innerHTML')获取元素内的全部HTML
* is_displayed()：设置该元素用户是否可见

In [92]:
element = driver.find_element_by_xpath('//a[@id="m_open"]')
element.click()
main_content = element.get_attribute('innerHTML')
main_content

'展开'

In [93]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [94]:
element = driver.find_element_by_xpath('//li[@title[contains(.,"素材管理")]]/a') 
# main_content = element.get_attribute('innerHTML')
# main_content
url2= element.get_attribute("href")
url2

'https://mp.weixin.qq.com/cgi-bin/appmsg?begin=0&count=10&t=media/appmsg_list&type=10&action=list&token=1217813269&lang=zh_CN'

In [95]:
driver.get(url2)

## 新建图文消息

In [96]:
element = driver.find_element_by_xpath('//*[text()[contains(.,"新建图文消息")]]') 
main_content = element.get_attribute('innerHTML')
main_content
element.click()

In [97]:
print (driver.window_handles)

['CDwindow-8A5E52CA7C83A2B6CC427D8854779C1C', 'CDwindow-99960EFF8A7B62348DB5966E3415FEE1']


In [98]:
# 新建图文消息开了另一分视窗，所以要切换 switch_to 
driver.switch_to.window(driver.window_handles[-1])

## 超链接

In [99]:
element = driver.find_element_by_xpath('//*[text()[contains(.,"超链接")]]') 
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

                超链接              


In [100]:
# 点 选择其他公众号
element = driver.find_element_by_xpath('//*[text()[contains(.,"选择其他公众号")]]') 
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

选择其他公众号


In [101]:
driver.find_element_by_xpath('//form//div[@class="inner_link_account_area"]//input[@class="weui-desktop-form__input"]').clear()
driver.find_element_by_xpath('//form//div[@class="inner_link_account_area"]//input[@class="weui-desktop-form__input"]').send_keys(公众号)

In [102]:
# 点放大镜搜
element = driver.find_element_by_xpath('//button[@class="weui-desktop-icon-btn weui-desktop-search__btn"]')
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

<div class="weui-desktop-icon weui-desktop-icon__inputSearch weui-desktop-icon__small"><!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <svg width="16" height="16" viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg"><path d="M11.33 10.007l4.273 4.273a.502.502 0 0 1 .005.709l-.585.584a.499.499 0 0 1-.709-.004L10.046 11.3a6.278 6.278 0 1 1 1.284-1.294zm.012-3.729a5.063 5.063 0 1 0-10.127 0 5.063 5.063 0 0 0 10.127 0z"></path></svg> <!----> <!----> <!----> <!----></div>


In [103]:
element = driver.find_element_by_xpath('//ul[@class="inner_link_account_list"]')
main_content = element.get_attribute('innerHTML')
print(main_content)
公众号SERP = main_content


<li class="inner_link_account_item"><div class="weui-desktop-vm_primary"><img src="http://mmbiz.qpic.cn/mmbiz_png/SXjwP8eTlZ6lPS2esEWzRvu11lkaMxib2cYiaib7cUTl6Zks8OwZov5khsibI0XWEJmFuWteFvAut3qcO22Mx4xiatQ/0?wx_fmt=png" class="inner_link_account_avatar"> <strong class="inner_link_account_nickname">中大南方文传体育部</strong> <i class="inner_link_account_wechat">微信号：zdnfwctyb</i></div> <div class="weui-desktop-vm_default inner_link_account_type">订阅号</div></li>


In [104]:
# 解析
root = fromstring(公众号SERP) 

In [105]:
主 = root.xpath('//li[@class="inner_link_account_item"]')

account_list = []
for e in 主:
    account_nickname = e.xpath('./div/strong[@class="inner_link_account_nickname"]')[0].text
    account_wechat = e.xpath('./div/i[@class="inner_link_account_wechat"]')[0].text
    account_img = e.xpath('./div/img/@src')[0]
    account = {"nickname": account_nickname, "wechat": account_wechat, "img": account_img,}
    account_list.append(account)

df_account = pd.DataFrame(account_list)

In [106]:
df_account

,nickname,wechat,img
0,中大南方文传体育部,微信号：zdnfwctyb,http://mmbiz.qpic.cn/mmbiz_png/SXjwP8eTlZ6lPS2...


In [107]:
element = driver.find_element_by_xpath('//ul[@class="inner_link_account_list"]/li')
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

<div class="weui-desktop-vm_primary"><img src="http://mmbiz.qpic.cn/mmbiz_png/SXjwP8eTlZ6lPS2esEWzRvu11lkaMxib2cYiaib7cUTl6Zks8OwZov5khsibI0XWEJmFuWteFvAut3qcO22Mx4xiatQ/0?wx_fmt=png" class="inner_link_account_avatar"> <strong class="inner_link_account_nickname">中大南方文传体育部</strong> <i class="inner_link_account_wechat">微信号：zdnfwctyb</i></div> <div class="weui-desktop-vm_default inner_link_account_type">订阅号</div>


In [108]:
# 跳转testing
'''
跳转_input = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/input')
跳转_a = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/a')
跳转_input.clear()
跳转_input.send_keys(2)
跳转_a.click()
'''

'\n跳转_input = driver.find_element_by_xpath(\'//span[@class="weui-desktop-pagination__form"]/input\')\n跳转_a = driver.find_element_by_xpath(\'//span[@class="weui-desktop-pagination__form"]/a\')\n跳转_input.clear()\n跳转_input.send_keys(2)\n跳转_a.click()\n'

In [109]:
# 跳转上限
l_e = driver.find_elements_by_xpath('//label[@class="weui-desktop-pagination__num"]')
l_e_int  = [int(x.text) for x in l_e] 
print (l_e_int)
print (l_e_int[0]==l_e_int[-1])

[1, 6]
False


In [110]:
pages = list(range(l_e_int[0],l_e_int[-1]+1 ))
#print(pages[0:2])
pages = list(range(1,l_e_int[-1]+1 ))
print(pages)

[1, 2, 3, 4, 5, 6]


## 循环/遍历

In [111]:
# global varialbes 
html_raw = dict()
main_content =""
element = None

In [112]:
def process_pages (pages):
    for p in pages:
        print (p,end='\t')

        跳转_input = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/input')
        跳转_a = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/a')
        跳转_input.clear()
        跳转_input.send_keys(p)
        跳转_a.click()

        time.sleep(45+120*random())

        element = driver.find_element_by_xpath('//div[@class="inner_link_article_list"]')
        main_content = element.get_attribute('innerHTML')
        #print(main_content)
        html_raw[p] = main_content

In [113]:
process_pages(pages)

1	2	3	4	5	6	

In [114]:
df = pd.DataFrame([html_raw]).T
df.columns = ["html_snippets"]
df

,html_snippets
1,"<div><label class=""inner_link_article_item""><s..."
2,"<div><label class=""inner_link_article_item""><s..."
3,"<div><label class=""inner_link_article_item""><s..."
4,"<div><label class=""inner_link_article_item""><s..."
5,"<div><label class=""inner_link_article_item""><s..."
6,"<div><label class=""inner_link_article_item""><s..."


In [115]:
%store html_raw
import pickle 
filehandler = open("html_raw", 'wb') 
pickle.dump(html_raw, filehandler)

Stored 'html_raw' (dict)


In [116]:
df_out = df[~df.duplicated()]
print (len(df_out))
df[df.duplicated()]

5


,html_snippets
6,"<div><label class=""inner_link_article_item""><s..."


In [117]:
try_again = list(df[df.duplicated()].index)
print(try_again)
try_again = try_again + list (set(pages).difference(set(df.index.values)))
try_again

[6]


[6]

## 暂存档

In [135]:
filename = fn ["output"] ["公众号_htm_snippets"] 
df_out.to_csv(filename.format(公众号=公众号), sep="\t", encoding="utf8")

In [121]:
def parse_html_snippets(_snippet_):
    root = fromstring(_snippet_) 
    title = [x.text for x in root.xpath('//div[@class="inner_link_article_title"]')]
    create_time = [x.text for x in root.xpath('//div[@class="inner_link_article_date"]')]
    link = [x for x in root.xpath('//a/@href')]
    _df_ = pd.DataFrame({"title":title, "create_time": create_time, "link":link})
    return(_df_)
    
l_df = []
for p in pages:
    _df_ = parse_html_snippets(df.loc[p,"html_snippets"])
    print (len(_df_), end=",")
    l_df.append(_df_)

5,5,5,5,6,5,

In [122]:
df_url_out = pd.concat(l_df).reset_index(drop=True)
df_url_out.loc[0:10]

,title,create_time,link
0,迎体育盛会，展文传风采——文学与传媒学院第七届体育嘉年华圆满落幕,2019-10-30,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
1,公示 | 文学与传媒学院第七届体育嘉年华各项比赛成绩及奖项,2019-10-29,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
2,焕新 | “ 害! 我们这不就来了吗！”,2019-10-21,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
3,赛程 | 拔河比赛晋级半决赛11强名单及赛程安排,2019-10-16,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
4,赛程 | 体育嘉年华预赛相关赛程安排,2019-10-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
5,第十一届“青出于篮”篮球专业赛,2018-04-20,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
6,第十一届“青出于篮”篮球专业赛,2018-04-18,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
7,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
8,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
9,第十一届“青出于篮”篮球专业赛||你来看了吗？,2018-04-13,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...


In [123]:
df_url_out.tail(5)

,title,create_time,link
26,第十一届“青出于篮”篮球专业赛,2018-04-20,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
27,第十一届“青出于篮”篮球专业赛,2018-04-18,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
28,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
29,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
30,第十一届“青出于篮”篮球专业赛||你来看了吗？,2018-04-13,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...


In [124]:
# tagging 标记
tagging_list = ["","篮球", "体育", "文传","WeChat",\
                "基建","新基建"] #overwritable

v_v_list = []

for tag in tagging_list:
    index_list = df_url_out [ df_url_out.title.str.contains(tag) ].index.tolist()
    v_v_pairs = pd.DataFrame({tag:index_list}).melt().set_index("value")
    v_v_list.append(v_v_pairs)

df_cat = v_v_list[0]
for d in v_v_list:
    df_cat.update(d)
    
# 尚未标记内容
df_url_out.loc [ df_cat.query('variable==""').index ]

,title,create_time,link
value,,,
2,焕新 | “ 害! 我们这不就来了吗！”,2019-10-21,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
3,赛程 | 拔河比赛晋级半决赛11强名单及赛程安排,2019-10-16,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
17,分享图片,2018-10-24,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
18,分享图片,2018-10-16,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
19,大件事啦！拔河比赛来啦！,2018-10-15,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
22,关于最有价值球员奖项的公告！,2018-05-18,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...


In [125]:
df_url_out[df_url_out.duplicated()]

,title,create_time,link
26,第十一届“青出于篮”篮球专业赛,2018-04-20,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
27,第十一届“青出于篮”篮球专业赛,2018-04-18,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
28,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
29,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
30,第十一届“青出于篮”篮球专业赛||你来看了吗？,2018-04-13,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...


In [126]:
df_url_out[~df_url_out.duplicated()]

,title,create_time,link
0,迎体育盛会，展文传风采——文学与传媒学院第七届体育嘉年华圆满落幕,2019-10-30,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
1,公示 | 文学与传媒学院第七届体育嘉年华各项比赛成绩及奖项,2019-10-29,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
2,焕新 | “ 害! 我们这不就来了吗！”,2019-10-21,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
3,赛程 | 拔河比赛晋级半决赛11强名单及赛程安排,2019-10-16,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
4,赛程 | 体育嘉年华预赛相关赛程安排,2019-10-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
5,第十一届“青出于篮”篮球专业赛,2018-04-20,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
6,第十一届“青出于篮”篮球专业赛,2018-04-18,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
7,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
8,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...
9,第十一届“青出于篮”篮球专业赛||你来看了吗？,2018-04-13,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...


In [127]:
df_o = df_url_out.join(df_cat).replace("", np.nan).fillna("无法分类")
df_o

,title,create_time,link,variable
0,迎体育盛会，展文传风采——文学与传媒学院第七届体育嘉年华圆满落幕,2019-10-30,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,文传
1,公示 | 文学与传媒学院第七届体育嘉年华各项比赛成绩及奖项,2019-10-29,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,体育
2,焕新 | “ 害! 我们这不就来了吗！”,2019-10-21,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,无法分类
3,赛程 | 拔河比赛晋级半决赛11强名单及赛程安排,2019-10-16,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,无法分类
4,赛程 | 体育嘉年华预赛相关赛程安排,2019-10-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,体育
5,第十一届“青出于篮”篮球专业赛,2018-04-20,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
6,第十一届“青出于篮”篮球专业赛,2018-04-18,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
7,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
8,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
9,第十一届“青出于篮”篮球专业赛||你来看了吗？,2018-04-13,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球


In [128]:
df_o[df_o.title.str.contains("篮球")]

,title,create_time,link,variable
5,第十一届“青出于篮”篮球专业赛,2018-04-20,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
6,第十一届“青出于篮”篮球专业赛,2018-04-18,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
7,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
8,第十一届“青出于篮”篮球专业赛,2018-04-14,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
9,第十一届“青出于篮”篮球专业赛||你来看了吗？,2018-04-13,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
10,第十二届“青出于篮”各专业篮球竞赛第四场,2019-04-06,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
11,第十二届“青出于篮”各专业篮球竞赛第三场,2019-03-29,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
12,第十二届“青出于篮”各专业篮球竞赛第二场,2019-03-27,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
13,第十二届“青出于篮”各专业篮球竞赛,2019-03-24,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球
14,第十二届“青出于蓝”篮球专业赛,2019-03-24,http://mp.weixin.qq.com/s?__biz=MzU5NDU0NTIzMQ...,篮球


In [129]:
df_stats = df_o.groupby(by="variable").agg({"title":"count"}).sort_values(by="title", ascending=False)
df_stats

,title
variable,
篮球,21
无法分类,6
体育,3
文传,1


## 输出

In [130]:
df_account.columns.name = "rel_accounts"
df_o.columns.name = "url_cat"
df_stats.columns.name = "stats"


In [131]:
_df_.columns.name

In [137]:
# Get the xlsxwriter workbook and worksheet objects.  
with pd.ExcelWriter(fn["output"]["公众号_xlsx"].format(公众号=公众号)) as writer:
    workbook  = writer.book

    for _df_ in [df_account, df_o, df_stats]:
        _df_.to_excel(writer, sheet_name = _df_.columns.name)
